# Import libraries

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from helpers import parse_variables, get_risk_level, hi_gauss_blob_risk_fun, blob_risk_fun, NW_risk_fun, square_risk_fun, map_to_color
from matplotlib.colors import LinearSegmentedColormap
import importlib.util
from k_means_constrained import KMeansConstrained


import warnings
warnings.filterwarnings("ignore")

import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
import tensorflow as tf

from helpers import parse_variables, get_risk_level, map_to_color, simulate_quant_trait
from models import ols_regression, manhattan_linear, gc
from deep_learning_models import abyss, deep_abyss

from scipy.stats import t
from scipy import stats
from scipy.stats import entropy

from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

from tensorflow.keras import Input, Model, layers, regularizers
from tensorflow.keras.layers import Input, Dense

2024-08-16 14:28:48.760606: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-16 14:28:48.775498: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-16 14:28:48.807720: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-16 14:28:48.860810: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-16 14:28:48.860899: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-16 14:28:48.896690: I tensorflow/core/platform/cpu_feature_guard.cc:

In [2]:
dict = parse_variables('geno_simulation.txt')
G = int(dict['G'])
L = int(dict['L'])
c = int(dict['c'])
k = int(dict['k'])
M = float(dict['M'])

# Thresholds
very_rare_threshold_L = float(dict['very_rare_threshold_L'])
very_rare_threshold_H = float(dict['very_rare_threshold_H'])

rare_threshold_L = float(dict['rare_threshold_L'])
rare_threshold_H = float(dict['rare_threshold_H'])

common_threshold_L = float(dict['common_threshold_L'])
common_threshold_H = float(dict['common_threshold_H'])

number_of_snps = (G*L)/2 # one loci per chromosome
number_of_individuals = c*k*k

In [3]:
very_rare = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}/genotype/02_veryrare_genotype_AF_{very_rare_threshold_L}_{very_rare_threshold_H}.pkl")
rare = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}/genotype/02_rare_genotype_AF_{rare_threshold_L}_{rare_threshold_H}.pkl")
common = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}/genotype/02_common_genotype_AF_{common_threshold_L}_{common_threshold_H}.pkl")

In [4]:
very_rare = very_rare.rename(columns=lambda x: 'VR' + x)/2
rare = rare.rename(columns=lambda x: 'R' + x)/2
common = common.rename(columns=lambda x: 'C' + x)/2
complete = pd.concat([common, rare, very_rare], axis=1)
complete = ((complete*2)-1)

# Run Abyss on LD block

In [5]:
def maf_prediction(bottle_in, geno_out, epoch, patience):
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(bottle_in, geno_out, test_size=0.2, random_state=42)
    
    # Regularization parameter
    l2_regularizer = 0.001
    
    # Original autoencoder model with L2 regularization
    decoder = tf.keras.Sequential([
        tf.keras.layers.Dense(150, activation='elu', input_shape=(bottle_in.shape[1],), kernel_regularizer=regularizers.l2(l2_regularizer)),  # First hidden layer with L2 regularization
        layers.BatchNormalization(),
        tf.keras.layers.Activation('elu'),
        tf.keras.layers.Dense(geno_out.shape[1], activation='linear', kernel_regularizer=regularizers.l2(l2_regularizer))  # Output layer
    ])
    
    # Compile the original model with L2 regularization
    decoder.compile(optimizer='adam',
                        loss='mean_squared_error',
                        metrics=['mean_absolute_error'])
    
    # Define Early Stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)
    
    # Fit the original model with Early Stopping
    history = decoder.fit(X_train, y_train, epochs=epoch, batch_size=32, validation_split=0.2, callbacks=[early_stopping], verbose=0)
    
    return decoder, history

In [6]:
path_output = f"data/G{G}_L{L}_c{c}_k{k}_M{M}/genotype/LD_blocks_estimated_mafs"
os.system(f"rm -rf {path_output}")
os.makedirs(path_output, exist_ok = True)

In [7]:
bottle_path = f"data/G{G}_L{L}_c{c}_k{k}_M{M}/phenotype/abyss_bottleneck"
bottle_file = [f for f in os.listdir(bottle_path) if f.split('_')[1] == 'bottleneck'][0]
bottle = pd.read_pickle(f"{bottle_path}/{bottle_file}")

In [8]:
clusters = bottle['cluster']
bottle.drop("cluster", axis=1, inplace=True)

In [9]:
path_one_hot_genotype = f"data/G{G}_L{L}_c{c}_k{k}_M{M}/genotype/LD_blocks_one_hot/"
path_lds = f"data/G{G}_L{L}_c{c}_k{k}_M{M}/genotype/LD_blocks"
ld_files = os.listdir(path_lds)


epoch = 200
patience = 15
p2s = []
twopqs = []
q2s = []
for ld_file in ld_files:
    db_minor = pd.read_pickle(f"{path_one_hot_genotype}{ld_file.split('.pkl')[0]}_db_minor.pkl")
    db_het = pd.read_pickle(f"{path_one_hot_genotype}{ld_file.split('.pkl')[0]}_db_het.pkl")
    db_major = pd.read_pickle(f"{path_one_hot_genotype}{ld_file.split('.pkl')[0]}_db_major.pkl")
    
    decoder, history = maf_prediction(bottle, db_major, epoch, patience)
    p2 = decoder(tf.convert_to_tensor(bottle, dtype=tf.float32))
    p2 = pd.DataFrame(data=p2, columns = db_major.columns)

    p2.index = db_major.index
    p2s.append(p2)

    decoder, history = maf_prediction(bottle, db_het, epoch, patience)
    twopq = decoder(tf.convert_to_tensor(bottle, dtype=tf.float32))
    twopq = pd.DataFrame(data=twopq, columns = db_het.columns)
    
    twopq.index = db_het.index
    twopqs.append(twopq)

    decoder, history = maf_prediction(bottle, db_minor, epoch, patience)
    q2 = decoder(tf.convert_to_tensor(bottle, dtype=tf.float32))
    q2 = pd.DataFrame(data=q2, columns = db_minor.columns)
    
    q2.index = db_minor.index
    q2s.append(q2)

In [10]:
path_output_global = f"data/G{G}_L{L}_c{c}_k{k}_M{M}/genotype/"
p2 = pd.concat(p2s, axis=1)
p2 = p2.sort_index()
p2 = p2[list(complete.columns)]

p2.to_pickle(f"{path_output_global}/esti_p2_zoom_0_via_esti_pop.pkl")

In [11]:
q2 = pd.concat(q2s, axis=1)
q2 = q2.sort_index()
q2 = q2[list(complete.columns)]
q2.to_pickle(f"{path_output_global}/esti_q2_zoom_0_via_esti_pop.pkl")

In [12]:
twopq = pd.concat(twopqs, axis=1)
twopq = twopq.sort_index()
twopq = twopq[list(complete.columns)]
twopq.to_pickle(f"{path_output_global}/esti_2pq_zoom_0_via_esti_pop.pkl")